In [2]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d /content/shape_predictor_68_face_landmarks.dat.bz2

--2023-03-28 17:06:45--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  52.8MB/s    in 1.2s    

2023-03-28 17:06:46 (52.8 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [3]:
model_path = '/content/shape_predictor_68_face_landmarks.dat'
predictor = dlib.shape_predictor(model_path)

In [5]:
# AAM 모델 구성
def build_aam_model(images):
    shapes = []
    for img in images:
        rects = detector(img, 1)
        shape = predictor(img, rects[0])
        shape = face_utils.shape_to_np(shape)
        shapes.append(shape)
    mean_shape = np.mean(shapes, axis=0)
    centered_shapes = shapes - mean_shape
    cov = np.cov(centered_shapes, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eigh(cov)
    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    p = eigenvectors.T.dot(centered_shapes.T)
    return mean_shape, eigenvectors, p


In [6]:
detector = dlib.get_frontal_face_detector()

In [25]:
# 얼굴 템플릿 이미지 다운로드
template_imgs = cv2.imread('/content/sel.jpg')
template_imgs2 = cv2.imread('/content/sel2.jpg')

template1 = cv2.cvtColor(template_imgs, cv2.COLOR_BGR2GRAY)
template2 = cv2.cvtColor(template_imgs2, cv2.COLOR_BGR2GRAY)

t1 = template1.reshape(1440 * 1080)
t2 = template2.reshape(1440 * 1080)
imgs = [t1, t2]
# # template_img = cv2.cvtColor(template_imgs ,  cv2.COLOR_BGR2GRAY)

# template_rects = detector(template_img, 1)
# template_shape = predictor(template_img, template_rects[0])
# template_shape = face_utils.shape_to_np(template_shape)


# template_imgs.shape

(1555200,)

In [26]:
mean_shape, eigenvectors, p = build_aam_model(imgs)

RuntimeError: ignored

In [98]:
test_img = cv2.imread('/content/sel2.jpg')
test_rects = detector(test_img, 1)
test_shape = predictor(test_img, test_rects[0])
test_shape = face_utils.shape_to_np(test_shape)


In [99]:
# AAM model fitting
feature_vector = np.zeros(eigenvectors.shape[0])
for i in range(len(feature_vector)):
    feature_vector[i] = np.dot(p[i], (test_shape - mean_shape).flatten())
reconstructed_shape = mean_shape + eigenvectors.dot(feature_vector)
M = cv2.estimateAffinePartial2D(reconstructed_shape, template_shape)
warped_img = cv2.warpAffine(test_img, M[0], (template_img.shape[1], template_img.shape[0]))

NameError: ignored